# Autonomous Agent Example

In this notebook, we instantiate an autonomous agent with an in-memory vectorstore. The agent is theoretically able to read the bibles.csv file in this repo, but your mileage may vary, and it probably makes sense to try turning bibles.csv into a vectorstore that the agent has access to as a tool.

## Paths to pursue

- [ ] Try to get the agent to read the bibles.csv file in this repo
- [ ] Try to get the agent to read a vectorstore that you create from bibles.csv
- [ ] Try to get the agent to identify translation rules from the Bibles in the eBible corpus (see [permissively licensed Bibles](https://github.com/sil-ai/sil-microsoft-hackathon-2023/blob/main/data/permissive_licensed_translations.csv))
- [ ] Try to get agents interacting in some kind of adversarial loop, where one agent is translating, and another is evaluating the translation, and the first agent is trying to improve its translation based on the evaluation

## Resources

- [AutoGPT agents in LangChain](https://python.langchain.com/docs/use_cases/more/agents/autonomous_agents/autogpt)
- [Combine agents with vectorstores](https://python.langchain.com/docs/modules/agents/how_to/agent_vectorstore)
- [LanceDB vectorstore (kind of like a NoSQL vectorstore)](https://python.langchain.com/docs/integrations/vectorstores/lancedb) & [API ref](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.lancedb.LanceDB.html)


In [18]:
!pip install --upgrade langchain faiss-cpu langchain_experimental -qq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
zsh:1: /usr/local/bin/pip: bad interpreter: /usr/local/opt/python@3.10/bin/python3.10: no such file or directory


In [3]:
import os

import getpass, os 
secret_key = getpass.getpass('Enter OpenAI secret key: ') 
os.environ['OPENAI_API_KEY'] = secret_key

# NOTE: For local model testing, use the following
# os.environ['OPENAI_API_KEY'] = 'empty'
# os.environ['OPENAI_API_BASE'] = 'http://192.168.1.76:8081/v1/'

In [4]:
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool



tools = [
    WriteFileTool(),
    ReadFileTool(),
]

In [5]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings

# Define your embedding model and set embedding size

# embeddings_model = OpenAIEmbeddings()
# embedding_size = 1536 # for openai embeddings

embeddings_model = HuggingFaceEmbeddings()
embedding_size = 768 # for huggingface embeddings

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Initialize the vectorstore as empty
import faiss

index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [16]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI
from langchain.memory.chat_message_histories import FileChatMessageHistory

agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0, verbose=True),
    memory=vectorstore.as_retriever(),
    chat_history_memory=FileChatMessageHistory("chat_history.txt"),
)

In [17]:
agent.run(["translate some Bible verses found in ../data/bibles.csv into Spanish"])

{
  "thoughts": {
    "text": "Since I have already reviewed my previous actions and checked for new tasks or objectives, I can move on to the next step. One of my goals is to translate some Bible verses into Spanish. To do this, I will need to access the Bible verses from the file 'bibles.csv' and then translate them using an online translation tool.",
    "reasoning": "By accessing the Bible verses from the file and using an online translation tool, I can efficiently translate them into Spanish.",
    "plan": "- Read the 'bibles.csv' file to access the Bible verses\n- Use an online translation tool to translate the verses into Spanish",
    "criticism": "I should ensure that the online translation tool I use is reliable and accurate to ensure the accuracy of the translations.",
    "speak": "I will read the 'bibles.csv' file to access the Bible verses and then use an online translation tool to translate them into Spanish."
  },
  "command": {
    "name": "read_file",
    "args": {
  

'I have finished reviewing my past actions and evaluating my performance.'